In [7]:
import io
import bs4
import pandas as pd

# Analyzing Income Statement Table with OpenAI
![img](./income_table.png)

## Retrieve Table from HTML

In [2]:
with open(r'ms_fy23_q1_html.htm','r') as f:
    content = f.read()
    soup = bs4.BeautifulSoup(content, 'html.parser')

In [3]:
table = soup.find('p',{'id':'INCOME_STATEMENTS'}).findNext('table')

In [4]:
table_txt = ''
for td in table:
    if td.text:
        table_txt+=(td.text)

In [5]:
print(table_txt.replace('\n',''))

(In millions, except per share amounts) (Unaudited) Three Months EndedDecember 31,  Six Months EndedDecember 31,                            2022 2021  2022  2021               Revenue:             Product $16,517 $20,779 $32,258 $37,410 Service and other  36,230  30,949  70,611  59,635                          Total revenue  52,747  51,728  102,869  97,045                       Cost of revenue:             Product  5,690  6,331  9,992  10,123 Service and other  11,798  10,629  22,948  20,483                       Total cost of revenue  17,488  16,960  32,940  30,606                       Gross margin  35,259  34,768  69,929  66,439 Research and development  6,844  5,758  13,472  11,357 Sales and marketing  5,679  5,379  10,805  9,926 General and administrative  2,337  1,384  3,735  2,671                       Operating income  20,399  22,247  41,917  42,485 Other income (expense), net  (60) 268  (6) 554                       Income before income taxes  20,339  22,515  41,911  43,039 Pr

In [11]:
## Try to read table using pandas
##(pd.read_html(str(table)))

## Connect to OpenAI

In [12]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-12-01"
openai.api_key = parser.get('openai_api','api_key')
model = parser.get('openai_api','api_model')

## Parse table into pandas dataframe

In [13]:
prompt = "Generate a pandas dataframe from the following table\nTable=\n###\n{}\n###".format(table_txt)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.2,  max_tokens=2000,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])
code  = response['choices'][0]['text']

Response:


df = pd.DataFrame({'Three Months Ended December 31, 2022': [16.517, 36.23, 52.747, 5.69, 11.798, 17.488, 35.259, 6.844, 5.679, 2.337, 20.399, -0.06, 20.339, 3.914, 16.425, 2.2, 2.2, 7.451, 7.473],
                    'Three Months Ended December 31, 2021': [20.779, 30.949, 51.728, 6.331, 10.629, 16.96, 34.768, 5.758, 5.379, 1.384, 22.247, 0.268, 22.515, 3.75, 18.765, 2.5, 2.48, 7.505, 7.555],
                    'Six Months Ended December 31, 2022': [32.258, 70.611, 102.869, 9.992, 22.948, 32.94, 69.929, 13.472, 10.805, 3.735, 41.917, -0.06, 41.911, 7.93, 33.981, 4.56, 4.54, 7.454, 7.479],
                    'Six Months Ended December 31, 2021': [37.41, 59.635, 97.045, 10.123, 20.483, 30.606, 66.439, 11.357, 9.926, 2.671, 42.485, 0.554, 43.039, 3.769, 39.27, 5.23, 5.19, 7.509, 7.561]},
                    index=['Revenue: Product', 'Revenue: Service and other', 'Total revenue', 'Cost of revenue: Product', 'Cost of revenue: Service and other', 'Total cost of revenue', 'Gros

In [14]:
exec(code)

In [15]:
df

,"Three Months Ended December 31, 2022","Three Months Ended December 31, 2021","Six Months Ended December 31, 2022","Six Months Ended December 31, 2021"
Revenue: Product,16.517,20.779,32.258,37.410
Revenue: Service and other,36.230,30.949,70.611,59.635
Total revenue,52.747,51.728,102.869,97.045
Cost of revenue: Product,5.690,6.331,9.992,10.123
Cost of revenue: Service and other,11.798,10.629,22.948,20.483
Total cost of revenue,17.488,16.960,32.940,30.606
Gross margin,35.259,34.768,69.929,66.439
Research and development,6.844,5.758,13.472,11.357
Sales and marketing,5.679,5.379,10.805,9.926
General and administrative,2.337,1.384,3.735,2.671


In [166]:
prompt = "Using the following dataframe generate python code to add year over year percentage change for each row:\ndata={}\n###\ndf = pd.DataFrame(data)\n".format(df.to_json)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.1,  max_tokens=2000,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])
code = response['choices'][0]['text']

Response:

df['YoY % Change'] = (df['Three Months Ended December 31, 2022'] - df['Three Months Ended December 31, 2021'])/df['Three Months Ended December 31, 2021']
df['YoY % Change'] = df['YoY % Change'].apply(lambda x: round(x*100,2))

df['YoY % Change_6M'] = (df['Six Months Ended December 31, 2022'] - df['Six Months Ended December 31, 2021'])/df['Six Months Ended December 31, 2021']
df['YoY % Change_6M'] = df['YoY % Change_6M'].apply(lambda x: round(x*100,2))


In [167]:
exec(code)

In [169]:
df

,"Three Months Ended December 31, 2022","Three Months Ended December 31, 2021","Six Months Ended December 31, 2022","Six Months Ended December 31, 2021",YoY % Change,YoY % Change_6M
Revenue: Product,16.517,20.779,32.258,37.410,-20.51,-13.77
Revenue: Service and other,36.230,30.949,70.611,59.635,17.06,18.41
Total revenue,52.747,51.728,102.869,97.045,1.97,6.00
Cost of revenue: Product,5.690,6.331,9.992,10.123,-10.12,-1.29
Cost of revenue: Service and other,11.798,10.629,22.948,20.483,11.00,12.03
Total cost of revenue,17.488,16.960,32.940,30.606,3.11,7.63
Gross margin,35.259,34.768,69.929,66.439,1.41,5.25
Research and development,6.844,5.758,13.472,11.357,18.86,18.62
Sales and marketing,5.679,5.379,10.805,9.926,5.58,8.86
General and administrative,2.337,1.384,3.735,2.671,68.86,39.84


In [173]:
prompt = "Using the following data generate code to identify top changes in percentages in data:\ndata={}\n###\ndf = pd.DataFrame(data)\n".format(df.to_json)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.1,  max_tokens=2000,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])
code = response['choices'][0]['text']

Response:

# Calculate the percentage change between the two periods
df['YoY % Change'] = (df['Three Months Ended December 31, 2022'] - df['Three Months Ended December 31, 2021']) / df['Three Months Ended December 31, 2021'] * 100
df['YoY % Change_6M'] = (df['Six Months Ended December 31, 2022'] - df['Six Months Ended December 31, 2021']) / df['Six Months Ended December 31, 2021'] * 100

# Sort the dataframe by the percentage change
df.sort_values(by='YoY % Change', ascending=False, inplace=True)

# Print the top 5 changes
print(df.head(5))


In [174]:
exec(code)

                                    Three Months Ended December 31, 2022  \
General and administrative                                         2.337   
Research and development                                           6.844   
Revenue: Service and other                                        36.230   
Cost of revenue: Service and other                                11.798   
Sales and marketing                                                5.679   

                                    Three Months Ended December 31, 2021  \
General and administrative                                         1.384   
Research and development                                           5.758   
Revenue: Service and other                                        30.949   
Cost of revenue: Service and other                                10.629   
Sales and marketing                                                5.379   

                                    Six Months Ended December 31, 2022  \
General and 

In [175]:
df

,"Three Months Ended December 31, 2022","Three Months Ended December 31, 2021","Six Months Ended December 31, 2022","Six Months Ended December 31, 2021",YoY % Change,YoY % Change_6M
General and administrative,2.337,1.384,3.735,2.671,68.858382,39.835268
Research and development,6.844,5.758,13.472,11.357,18.860716,18.622876
Revenue: Service and other,36.230,30.949,70.611,59.635,17.063556,18.405299
Cost of revenue: Service and other,11.798,10.629,22.948,20.483,10.998212,12.034370
Sales and marketing,5.679,5.379,10.805,9.926,5.577245,8.855531
Provision for income taxes,3.914,3.750,7.930,3.769,4.373333,110.400637
Total cost of revenue,17.488,16.960,32.940,30.606,3.113208,7.625956
Total revenue,52.747,51.728,102.869,97.045,1.969920,6.001340
Gross margin,35.259,34.768,69.929,66.439,1.412218,5.252939
Weighted average shares outstanding: Basic,7.451,7.505,7.454,7.509,-0.719520,-0.732454
